In [0]:
emp_data =  [
(1,'manish',26,20000,'india','IT'),
(2,'rahul',None,40000,'germany','engineering'),
(3,'pawan',12,60000,'india','sales'),
(4,'roshini',44,None,'uk','engineering'),
(5,'raushan',35,70000,'india','sales'),
(6,None,29,200000,'uk','IT'),
(7,'adam',37,65000,'us','IT'),
(8,'chris',16,40000,'us','sales'),
(None,None,None,None,None,None),
(7,'adam',37,65000,'us','IT')
]


In [0]:
emp_schema = ['id','Name','age','Country','department']

In [0]:
emp_df = spark.createDataFrame(emp_data, emp_schema)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
#First Used the withColumn to create a new column
# Then we used the when otherwise condition which is what is used as if else in pyspark
emp_df.withColumn ("adult",when (col ("age") <18, "No")
.when (col ("age") >18, "Yes")
.otherwise ("Novalue")) .show()
                        
                         

+----+-------+----+-------+----------+-----------+-------+
|  id|   Name| age|Country|department|         _6|  adult|
+----+-------+----+-------+----------+-----------+-------+
|   1| manish|  26|  20000|     india|         IT|    Yes|
|   2|  rahul|null|  40000|   germany|engineering|Novalue|
|   3|  pawan|  12|  60000|     india|      sales|     No|
|   4|roshini|  44|   null|        uk|engineering|    Yes|
|   5|raushan|  35|  70000|     india|      sales|    Yes|
|   6|   null|  29| 200000|        uk|         IT|    Yes|
|   7|   adam|  37|  65000|        us|         IT|    Yes|
|   8|  chris|  16|  40000|        us|      sales|     No|
|null|   null|null|   null|      null|       null|Novalue|
|   7|   adam|  37|  65000|        us|         IT|    Yes|
+----+-------+----+-------+----------+-----------+-------+



In [0]:
#Modified the above code to remove null values and replace tgem with 19
emp_df.withColumn ("age"
,when (col ("age") .isNull(),lit(19))
.otherwise(col("age")))\
.withColumn ("adult" ,when (col ("age") >18, "Yes")
.otherwise("No")) .show()

+----+-------+---+-------+----------+-----------+-----+
|  id|   Name|age|Country|department|         _6|adult|
+----+-------+---+-------+----------+-----------+-----+
|   1| manish| 26|  20000|     india|         IT|  Yes|
|   2|  rahul| 19|  40000|   germany|engineering|  Yes|
|   3|  pawan| 12|  60000|     india|      sales|   No|
|   4|roshini| 44|   null|        uk|engineering|  Yes|
|   5|raushan| 35|  70000|     india|      sales|  Yes|
|   6|   null| 29| 200000|        uk|         IT|  Yes|
|   7|   adam| 37|  65000|        us|         IT|  Yes|
|   8|  chris| 16|  40000|        us|      sales|   No|
|null|   null| 19|   null|      null|       null|  Yes|
|   7|   adam| 37|  65000|        us|         IT|  Yes|
+----+-------+---+-------+----------+-----------+-----+



In [0]:
# Adding the "age_wise" column to seperate the age groups into 3 brackets 
emp_df = emp_df.withColumn(
    "age_wise",
    when((col("age") > 0) & (col("age") < 18), "Minor")
    .when((col("age") > 18) & (col("age") < 30), "Mid")
    .otherwise("Adult")
)

emp_df.show()


+----+-------+----+-------+----------+-----------+--------+
|  id|   Name| age|Country|department|         _6|age_wise|
+----+-------+----+-------+----------+-----------+--------+
|   1| manish|  26|  20000|     india|         IT|     Mid|
|   2|  rahul|null|  40000|   germany|engineering|   Adult|
|   3|  pawan|  12|  60000|     india|      sales|   Minor|
|   4|roshini|  44|   null|        uk|engineering|   Adult|
|   5|raushan|  35|  70000|     india|      sales|   Adult|
|   6|   null|  29| 200000|        uk|         IT|     Mid|
|   7|   adam|  37|  65000|        us|         IT|   Adult|
|   8|  chris|  16|  40000|        us|      sales|   Minor|
|null|   null|null|   null|      null|       null|   Adult|
|   7|   adam|  37|  65000|        us|         IT|   Adult|
+----+-------+----+-------+----------+-----------+--------+



In [0]:
emp_df.createTempView("table")

In [0]:
# Creating the new column in spark sql where case when is used as the if else statement
spark.sql("""
select *,
case when age<18 then 'minor' when age>18 then
'major'
else 'novalue' end as adult

from table
""").show()

+----+-------+----+-------+----------+-----------+--------+-------+
|  id|   Name| age|Country|department|         _6|age_wise|  adult|
+----+-------+----+-------+----------+-----------+--------+-------+
|   1| manish|  26|  20000|     india|         IT|     Mid|  major|
|   2|  rahul|null|  40000|   germany|engineering|   Adult|novalue|
|   3|  pawan|  12|  60000|     india|      sales|   Minor|  minor|
|   4|roshini|  44|   null|        uk|engineering|   Adult|  major|
|   5|raushan|  35|  70000|     india|      sales|   Adult|  major|
|   6|   null|  29| 200000|        uk|         IT|     Mid|  major|
|   7|   adam|  37|  65000|        us|         IT|   Adult|  major|
|   8|  chris|  16|  40000|        us|      sales|   Minor|  minor|
|null|   null|null|   null|      null|       null|   Adult|novalue|
|   7|   adam|  37|  65000|        us|         IT|   Adult|  major|
+----+-------+----+-------+----------+-----------+--------+-------+

